In [ ]:
!pip install --no-cache-dir --upgrade jupyter

In [ ]:
from sgp4.api import Satrec
from datetime import datetime, timedelta
import numpy as np
from scipy.spatial.transform import Rotation as R

from database.db_handler import create_db, iniciar_simulacao, salvar_dado_orbital

# Passo 1 - Inicializar o banco
create_db()
sim_id = iniciar_simulacao("Importação via TLE ISS")

# TLE da ISS (pode trocar por outro satélite depois)
line1 = '1 25544U 98067A   24106.45347222  .00012190  00000+0  21377-3 0  9992'
line2 = '2 25544  51.6414  52.8843 0003046  96.6780  42.0736 15.51762384294959'
sat = Satrec.twoline2rv(line1, line2)

def datetime_to_julian_date(dt):
    year, month, day = dt.year, dt.month, dt.day
    hour, minute = dt.hour, dt.minute
    second = dt.second + dt.microsecond / 1e6
    jd = 367 * year - int((7 * (year + int((month + 9) / 12))) / 4) \
         + int((275 * month) / 9) + day + 1721013.5 \
         + (hour + minute / 60 + second / 3600) / 24
    fr = (hour + minute / 60 + second / 3600) / 24 % 1
    return jd, fr

# Parâmetros da simulação
base_time = datetime.utcnow()
interval = 30  # segundos
samples = 200

for i in range(samples):
    dt = base_time + timedelta(seconds=i * interval)
    jd, fr = datetime_to_julian_date(dt)
    error_code, position, velocity = sat.sgp4(jd, fr)

    if error_code != 0:
        print(f"Erro na propagação: {dt} - Código {error_code}")
        continue

    r = np.array(position)
    v = np.array(velocity)

    # Base para orientação
    z = -r / np.linalg.norm(r)
    y = np.cross(z, v)
    y /= np.linalg.norm(y)
    x = np.cross(y, z)
    rot_matrix = np.vstack([x, y, z]).T
    rotation = R.from_matrix(rot_matrix)

    # Quaternions e Euler
    quat = rotation.as_quat()  # w, x, y, z
    euler = rotation.as_euler('xyz', degrees=True)

    salvar_dado_orbital(
        sim_id,
        i * interval,
        r[0], r[1], r[2],
        v[0], v[1], v[2],
        euler[0], euler[1], euler[2],
        tipo="real"  # ou "simulado", "hibrido"
    )

print("✅ Dados salvos no banco gyroai.db!")